# Support for Quantum ESPRESSO in exatomic

In [13]:
import pandas as pd
import exatomic
from exatomic import qe
from exatomic.base import resource

In [9]:
exatomic.Editor(resource("qe.inp"))    # Visualize the example input file if desired

 0: &control 
 1:   calculation = 'cp'
 2:   restart_mode = 'restart'
 3:   pseudo_dir = './'
 4:   outdir = './'
 5:   ndr = 52
 6:   ndw = 53
 7:   nstep = 100000
 8:   isave = 100
 9:   dt = 5.0
10:   tprnfor = .true.
11:   etot_conv_thr = 1.d-10
12:   ekin_conv_thr = 1.d-10
13: /
14: &system
15:   ibrav = 1
16:   celldm(1) = 8.992926526086501
17:   nat = 24
18:   ntyp = 2
19:   ecutwfc = 100.0
20:   nr1b = 20
21:   nr2b = 20
22:   nr3b = 20
23: /
24: &electrons 
25:   ortho_eps = 1.d-10
26:   conv_thr = 1.d-10
27:   electron_dynamics = 'verlet'
28:   electron_temperature = 'not_controlled'
29: /
...
32:   ion_temperature = 'not_controlled'
33: /
34: ATOMIC_SPECIES
35: H  2.01355d0   H.pbe-rrkjus_psl.1.0.0.UPF
36: O  15.9994d0   O.pbe-n-rrkjus_psl.1.0.0.UPF
37: ATOMIC_POSITIONS (angstrom)
38: H        0.546318147   3.590197100   1.378263094
39: H        0.954517747   1.514354410   2.962342290
40: H        0.787888461   0.839350586   0.882540414
41: H        0.690257635   4.012130108

In [10]:
xyz = qe.parse_symbols_from_input(resource("qe.inp"))
xyz

,symbol,x,y,z
0,H,1.032386,6.784453,2.604526
1,H,1.803768,2.861700,5.597986
2,H,1.488886,1.586134,1.667751
3,H,1.304391,7.581787,6.658065
4,H,2.687215,5.624346,7.935414
5,H,3.611814,0.867768,6.808235
6,H,3.455902,3.536857,1.089975
7,H,3.677819,7.840515,2.902969
8,H,5.150307,4.829348,3.958955
9,H,5.359657,1.344457,3.427006


In [53]:
symbols = xyz['symbol'].tolist()
pos = qe.parse_xyz(resource("qe.pos"), symbols)
vel = qe.parse_xyz(resource("qe.vel"), symbols)
vel.columns = ("vx", "vy", "vz", "frame", "symbol")
force = qe.parse_xyz(resource("qe.for"), symbols)
force.columns = ("fx", "fy", "fz", "frame", "symbol")

In [54]:
pos.head()

,x,y,z,frame,symbol
0,1.386029,10.148803,2.527119,86300,H
1,6.298432,0.408980,6.614086,86300,H
2,2.705972,4.812221,1.851418,86300,H
3,0.925476,7.572369,3.711328,86300,H
4,-0.712621,7.029148,6.158907,86300,H


In [55]:
vel.head()

,vx,vy,vz,frame,symbol
0,0.000266,0.000348,-0.000371,86300,H
1,-0.000284,-0.000844,0.000613,86300,H
2,0.000434,-0.000134,0.000023,86300,H
3,-0.000236,0.001189,0.000255,86300,H
4,0.000289,-0.000445,0.000056,86300,H


In [56]:
force.head()

,fx,fy,fz,frame,symbol
0,-0.014144,-0.001770,-0.010127,86300,H
1,-0.003695,0.015956,-0.013014,86300,H
2,-0.003526,-0.001231,0.003031,86300,H
3,0.017270,-0.005110,-0.001918,86300,H
4,0.002262,0.006420,0.002992,86300,H


In [57]:
atom = pd.concat((pos, vel[["vx", "vy", "vz"]], force[["fx", "fy", "fz"]]), axis=1)

In [58]:
# Some extra steps to set the column names
evp = qe.parse_evp(resource("qe.evp"), symbols)
columns = evp.iloc[0, :].tolist()[1:] + ["None"]    # Add a dummy column
evp = evp.drop("#", axis=0)                         # Delete first row
evp.columns = columns
evp['atom_count'] = len(symbols)
evp = evp.dropna(how='all', axis=1)                 # Remove dummy column

In [59]:
evp.head()

,time(ps),ekinc,T_cell(K),Tion(K),etot,enthal,econs,econt,Volume,Pressure(GPa),atom_count
86300,1.043756E+01,1.165034E-03,0.000000E+00,1.991763E+02,-139.02493778,-139.02493778,-139.00223062,-139.04934698,7.272825E+02,-0.00000,24
86310,1.043877E+01,1.177623E-03,0.000000E+00,1.951384E+02,-139.02452520,-139.02452520,-139.00227838,-139.00110075,7.272825E+02,-0.00000,24
86320,1.043998E+01,2.023734E-03,0.000000E+00,1.817591E+02,-139.02385374,-139.02385374,-139.00313223,-139.00110849,7.272825E+02,-0.00000,24
86330,1.044119E+01,1.223379E-03,0.000000E+00,1.886373E+02,-139.02383875,-139.02383875,-139.00233309,-139.00110971,7.272825E+02,-0.00000,24
86340,1.044240E+01,2.015737E-03,0.000000E+00,1.967750E+02,-139.02555726,-139.02555726,-139.00312386,-139.00110813,7.272825E+02,-0.00000,24


In [60]:
# Make a universe
uni = exatomic.Universe(atom=atom, frame=evp)

In [61]:
len(uni)

10001

In [62]:
uni.atom.dtypes

x          float64
y          float64
z          float64
frame     category
symbol    category
vx         float64
vy         float64
vz         float64
fx         float64
fy         float64
fz         float64
dtype: object

In [63]:
%time uni.compute_atom_two()

CPU times: user 4.69 s, sys: 609 ms, total: 5.3 s
Wall time: 5.35 s


In [64]:
uni.atom.shape

(240024, 11)

In [65]:
uni.atom_two.shape

(1874130, 4)

In [66]:
uni.molecule.shape

(80030, 3)

In [ ]:
exatomic.UniverseWidget(uni)